In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
import tensorflow_datasets as tfds

In [3]:
def scale_images(data): 
    image = data['image']
    return image / 255

In [4]:
# Reload the dataset 
ds = tfds.load('fashion_mnist', split='train')
# Running the dataset through the scale_images preprocessing step
ds = ds.map(scale_images) 
# Cache the dataset for that batch 
ds = ds.cache()
# Shuffle it up 
ds = ds.shuffle(60000)
# Batch into 128 images per sample
ds = ds.batch(128)
# Reduces the likelihood of bottlenecking 
ds = ds.prefetch(64)

In [5]:
# Bring in the sequential api for the generator and discriminator
from tensorflow.keras.models import Sequential
# Bring in the layers for the neural network
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D

In [6]:
def build_generator(): 
    model = Sequential()
    
    # Takes in random values and reshapes it to 7x7x128
    # Beginnings of a generated image
    model.add(Dense(7*7*128, input_dim=128))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((7,7,128)))
    
    # Upsampling block 1 
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Upsampling block 2 
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Convolutional block 1
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Convolutional block 2
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Conv layer to get to one channel
    model.add(Conv2D(1, 4, padding='same', activation='sigmoid'))
    
    return model

In [7]:
def build_discriminator(): 
    model = Sequential()
    
    # First Conv Block
    model.add(Conv2D(32, 5, input_shape = (28,28,1)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Second Conv Block
    model.add(Conv2D(64, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Third Conv Block
    model.add(Conv2D(128, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Fourth Conv Block
    model.add(Conv2D(256, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Flatten then pass to dense layer
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    return model 

In [8]:
# Adam is going to be the optimizer for both
from tensorflow.keras.optimizers import Adam
# Binary cross entropy is going to be the loss for both 
from tensorflow.keras.losses import BinaryCrossentropy

In [9]:
g_opt = Adam(learning_rate=0.0001) 
d_opt = Adam(learning_rate=0.00001) 
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

In [10]:
from tensorflow.keras.models import Model

class FashionGAN(Model): 
    def __init__(self, generator, discriminator, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.generator = generator 
        self.discriminator = discriminator 
        
    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs): 
        super().compile(*args, **kwargs)
        
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss 

    def train_step(self, batch):
        real_images = batch
        fake_images = self.generator(tf.random.normal((128, 128, 1)), training=False)
        
        with tf.GradientTape() as d_tape: 
            yhat_real = self.discriminator(real_images, training=True) 
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            
            noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
            
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
        
        with tf.GradientTape() as g_tape: 
            gen_images = self.generator(tf.random.normal((128,128,1)), training=True)
            predicted_labels = self.discriminator(gen_images, training=False)
                                        
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) 
            
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
        
        return {"d_loss":total_d_loss, "g_loss":total_g_loss}

In [14]:
# Initialize the strategy
strategy = tf.distribute.MirroredStrategy()

# Use the strategy scope to create and compile the models
with strategy.scope():
    # Build the generator and discriminator models
    generator = build_generator()
    discriminator = build_discriminator()

    # Define the optimizers inside the strategy scope
    g_opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    d_opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

    # Define the loss functions
    g_loss = tf.keras.losses.BinaryCrossentropy()
    d_loss = tf.keras.losses.BinaryCrossentropy()

    # Initialize and compile the GAN model
    gan = FashionGAN(generator, discriminator)
    gan.compile(g_opt=g_opt, d_opt=d_opt, g_loss=g_loss, d_loss=d_loss)

In [15]:
hist = gan.fit(ds, epochs=20)

Epoch 1/20


2024-08-24 06:29:37.765311: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_186/cond/StatefulPartitionedCall/replica_1/sequential_3_1/dropout_5_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


469/469 ━━━━━━━━━━━━━━━━━━━━ 100s 166ms/step - d_loss: 0.5565 - g_loss: 1.1821
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 78s 166ms/step - d_loss: 0.5351 - g_loss: 2.6954
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 169ms/step - d_loss: 0.4563 - g_loss: 4.4820
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 170ms/step - d_loss: 0.4692 - g_loss: 4.1646
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 170ms/step - d_loss: 0.4440 - g_loss: 4.0367
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 170ms/step - d_loss: 0.4576 - g_loss: 3.6683
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 170ms/step - d_loss: 0.4571 - g_loss: 3.4835
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 171ms/step - d_loss: 0.6049 - g_loss: 1.1761
Epoch 9/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 170ms/step - d_loss: 0.5785 - g_loss: 1.0496
Epoch 10/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 170ms/step - d_loss: 0.4794 - g_loss: 0.8526
Epoch 11/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 170ms/step - d_loss: 0.4909 - g_loss: 0.8688
Epoch 12/20
469/469 ━━━━━━━━━